<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/07_10_22_SBERT_NN_Gossipcop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 1.8 MB 36.7 MB/s 
     |████████████████████████████████| 4.4 MB 57.0 MB/s 
     |████████████████████████████████| 1.2 MB 61.0 MB/s 
     |████████████████████████████████| 101 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 6.6 MB 62.5 MB/s 
     |████████████████████████████████| 145 kB 2.4 MB/s 
     |████████████████████████████████| 181 kB 69.8 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9fa7fdc4c6b765e37ece2ec3e2385ce3bc8c8d609feeeec354bc6b11db8b6b59
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
  Created wheel for pathtools: filename

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import json

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output

In [3]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return score_func.__name__.split("_")[0]

Load data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/"gossipcop_agg.csv", index_col=0, converters=converters)
df.head()

,text,tweets,retweets,label,url
title,,,,,
Kendall Kylie Jenner Jenner NOT Upset Upset Over Caitlyn Engagement Engagement Despite Report,NaN,[],"[995423424741888001, 995461685166202880, 99987...",fake,NaN
Kim Kardashian Dethroned Dethroned By Khloe As Hot Hot Sister,NaN,[],"[848843565027516416, 849030801970868224, 84884...",fake,NaN
Kim Kardashian Did NOT Hot Staffer Hot Staffer To Save Marriage Save Marriage Despite Claim,NaN,[],"[940685393112064001, 977921622672920576, 94031...",fake,NaN
The Voice The Voice Team NOT Surprised Surprised Adam Levine Behati Prinsloo Pregnant Again Before Blake Shelton Gwen Stefani,NaN,[],[],fake,NaN
Drake NOT Angelina Jolie s Toy Boy Toy Boy Despite Late And Wrong Report,NaN,"[{'id': 948630026496323585, 'text': 'Drake NOT...","[948022124626808832, 948630026496323585, 94801...",fake,NaN


In [10]:
titles = df.title.tolist()
texts = (df.title + "\n" + df.text).tolist()

AttributeError: ignored

In [12]:
len(texts)

894

Compute Embeddings

In [13]:
title_embedding_file = base_dir/"gossipcop_sbert_title_embeddings.npy"

if title_embedding_file.exists():
    title_embeddings = np.load(title_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)

    title_embeddings = model.encode(titles, show_progress_bar=True)
    np.save(title_embedding_file, title_embeddings)

In [14]:
text_embedding_file = base_dir/"gossipcop_sbert_fulltext_embeddings.npy"

if text_embedding_file.exists():
    text_embeddings = np.load(text_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)

    text_embeddings = model.encode(texts, show_progress_bar=True)
    np.save(text_embedding_file, text_embeddings)

Training

In [16]:
import tensorflow as tf

In [17]:
def get_model(d_h=64, p_in:float=0.2, p_out:float=0.2):
    model = tf.keras.Sequential([
        tf.keras.layers.Dropout(p_in), # input dropout
        tf.keras.layers.Dense(d_h, activation='relu'),
        tf.keras.layers.Dropout(p_out),  
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  optimizer='adam',metrics=['accuracy'])
    return model

Title embeddings

In [18]:
X = title_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 768), (19968,))

In [19]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [20]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model()
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    logits = model.predict(X_test).squeeze()
    y_pred = (logits > 0.).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

In [21]:
params = {
    "epochs":200,
    "bs":128,
    # 'lr':1e-3
}

In [22]:
GROUP = f"gossipcop-title-sbert-mlp"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/200
125/125 [==============================] - 5s 8ms/step - loss: 0.4767 - accuracy: 0.7480 - val_loss: 0.4163 - val_accuracy: 0.8032 - _timestamp: 1657488177.0000 - _runtime: 22.0000
Epoch 2/200
125/125 [==============================] - 1s 7ms/step - loss: 0.3943 - accuracy: 0.8195 - val_loss: 0.3956 - val_accuracy: 0.8172 - _timestamp: 1657488178.0000 - _runtime: 23.0000
Epoch 3/200
125/125 [==============================] - 1s 6ms/step - loss: 0.3786 - accuracy: 0.8277 - val_loss: 0.3862 - val_accuracy: 0.8167 - _timestamp: 1657488179.0000 - _runtime: 24.0000
Epoch 4/200
125/125 [==============================] - 1s 6ms/step - loss: 0.3723 - accuracy: 0.8304 - val_loss: 0.3818 - val_accuracy: 0.8247 - _timestamp: 1657488180.0000 - _runtime: 25.0000
Epoch 5/200
125/125 [==============================] - 1s 5ms/step - loss: 0.3655 - accuracy: 0.8346 - val_loss: 0.3784 - val_accuracy: 0.8280 - _timestamp: 1657488180.0000 - _runtime: 25.0000
Epoch 6/200
125/125 [==============

accuracy,▁▂▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1,▁
loss,█▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁
recall,▁
val_accuracy,▁▄▅▇████▆█▇▆▆▅▅▅▆▆▅▃▆▅▅▃▆▅▅▃▄▃▄▅▄▃▃▄▄▆▆▂
val_loss,▃▂▂▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
accuracy,0.84702
best_epoch,25
best_val_loss,0.34969


Text Embeddings

In [23]:
X = text_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 768), (19968,))

In [24]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [25]:
GROUP = f"gossipcop-fulltext-sbert-mlp"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

wandb: Currently logged in as: saloni (saloniteam). Use `wandb login --relogin` to force relogin


wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/200
125/125 [==============================] - 1s 6ms/step - loss: 0.4739 - accuracy: 0.7473 - val_loss: 0.4088 - val_accuracy: 0.8197 - _timestamp: 1657488326.0000 - _runtime: 4.0000
Epoch 2/200
125/125 [==============================] - 1s 4ms/step - loss: 0.3866 - accuracy: 0.8247 - val_loss: 0.3890 - val_accuracy: 0.8192 - _timestamp: 1657488327.0000 - _runtime: 5.0000
Epoch 3/200
125/125 [==============================] - 1s 5ms/step - loss: 0.3726 - accuracy: 0.8355 - val_loss: 0.3820 - val_accuracy: 0.8340 - _timestamp: 1657488327.0000 - _runtime: 5.0000
Epoch 4/200
125/125 [==============================] - 1s 4ms/step - loss: 0.3643 - accuracy: 0.8374 - val_loss: 0.3774 - val_accuracy: 0.8340 - _timestamp: 1657488328.0000 - _runtime: 6.0000
Epoch 5/200
125/125 [==============================] - 1s 4ms/step - loss: 0.3589 - accuracy: 0.8401 - val_loss: 0.3732 - val_accuracy: 0.8403 - _timestamp: 1657488328.0000 - _runtime: 6.0000
Epoch 6/200
125/125 [===================

accuracy,▁▂▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇███████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1,▁
loss,█▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
precision,▁
recall,▁
val_accuracy,▁▅▅▇█▆▇▇▇▇▇█▇▇▇▇▇▆▇▇▇▇▇▇█▇███▆▇▇▇▇▆▇▆▆▇▆
val_loss,▅▃▂▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▄▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
accuracy,0.85653
best_epoch,30
best_val_loss,0.3426
